# Example Inference
In this Notebook, we illustrate an instance of model-based inference. This illustration entails the following steps:

1. Loading a sample query, which pertains to an "offer."
2. Conducting a search operation, which involves retrieval and ranking, to identify the most closely matching product corresponding to the given query.

The retrieval process employs a Siamese network in conjunction with an approximate neighbor search mechanism to obtain the top 10 products that exhibit the highest degree of similarity.

As for the ranking process, it is worth noting that, as of now, an alternative machine learning model is not yet available. Therefore, from the pool of retrieved products, the system selects the product whose price falls within the expected range, determined by the price calculations associated with the respective product.

In [2]:
from offerexpert.inference import Inference
from pathlib import Path
import json
from offerexpert.data.provider.load_data import PATH_DATA
from offerexpert.evaluate import isin_expected_range

_ARTEFACT_FOLDER = Path("../models/experiment-default")

## Load query

In [3]:
with open("query.json", "r", encoding="utf-8") as fp:
    offer = json.load(fp)
print(offer)

{'offerId': '101551_15530-xiaomi-pocophone-f1-global-version-618-inch-6gb-128gb-snapdragon-845-octa-core-4g-smartphone', 'name': 'xiaomi pocophone f1 global version 6.18 inch 6gb 128gb snapdragon 845 octa core 4g smartphone', 'description': None, 'gtin14': None, 'isbn10': None, 'priceAmount': '309.99', 'lastUpdated': '2019-09-13T07:35:37.408+02:00'}


## Retrieval: Match with the most similar Product IDs

In [4]:
model_inference = Inference.load_model(_ARTEFACT_FOLDER)
matched_product_ids = model_inference(offer["name"], top_n=10)
print(f"matched_product_ids: {matched_product_ids}")

matched_product_ids: ['4689772' '4689772' '4689772' '4689772' '4689772' '4689772' '4689772'
 '4689772' '4689772' '4689772']


## Ranker: Select by amount

In [5]:
with open(Path(PATH_DATA) / "products.json", "r", encoding="utf-8") as fp:
    products = json.load(fp)

for product_id in matched_product_ids:
    amount_mean_std = [
        (p["priceCalculations"].get("mean", 0.0), p["priceCalculations"].get("standardDeviation", 0.0)) if isinstance(
            p["priceCalculations"], dict) else (0.0, 0.0) for p in products if p["id"] == product_id]
    mean_ = amount_mean_std[0][0]
    std_ = amount_mean_std[0][1]
    if isin_expected_range(offer["priceAmount"], mean_, std_):
        matched_product_id = product_id
        break
else:
    matched_product_id = matched_product_ids[0]
print(f"matched product id: {matched_product_id}")

matched product id: 4689772


## Check matched product

In [6]:
for product in products:
    if product["id"] == matched_product_id:
        verified_names = product["positivelyVerifiedOfferNames"]
        if verified_names:
            print(",".join([p["name"] for p in verified_names]))

xiaomi poco f1 128gb dual sim / unlocked - blue,xiaomi pocophone f1 128gb blue ds eu spec - 24 month warranty,movil smartphone xiaomi pocophone f1 6gb 128gb azul,xiaomi pocophone f1 global version 6.18 inch 6gb 128gb snapdragon 845 octa core 4g smartphone,xiaomi pocophone f1 6gb ram og 128gb storage (dual sim)-uk sim-gratis smartphone-6,18-tommer android 8,1-grafit sort,xiaomi poco f1 128gb dual sim / unlocked - blue,xiaomi pocophone f1 global version 6.18 inch 6gb 128gb snapdragon 845 octa core 4g smartphone,xiaomi pocophone f1 dual sim 128gb 6gb ram black,movil smartphone xiaomi pocophone f1 6gb 128gb negro,xiaomi poco f1 128gb dual sim / unlocked - black,xiaomi pocophone f1 6gb/128gb dual sim sim free/ unlocked - black,xiaomi pocophone f1 dual sim 128gb 6gb ram sort,xiaomi poco f1 128gb dual sim / unlocked - black,xiaomi pocophone f1 6.18 128gb 4g sort,xiaomi pocophone f1 (6.18 inch) 128gb 12mp smartphone (graphite black),xiaomi m1805e10an-128 pocophonef1 smartphone 128 gb - p, 64gb